In [1]:
request = {"body": "{\"length\":\"9.0\",\"diameter\":\"5.0\",\"height\":\"3.8\",\"wholeweight\":\"1.5\",\"shuckedweight\":\"5.9\",\"visceraweight\":\"3.0\",\"shellweight\":\"1.5\",\"female\":\"1\",\"infant\":\"0\",\"male\":\"0\"}"}

In [2]:
request

{'body': '{"length":"9.0","diameter":"5.0","height":"3.8","wholeweight":"1.5","shuckedweight":"5.9","visceraweight":"3.0","shellweight":"1.5","female":"1","infant":"0","male":"0"}'}

In [3]:
import json

body = list(json.loads(request['body']).values())
body

['9.0', '5.0', '3.8', '1.5', '5.9', '3.0', '1.5', '1', '0', '0']

In [4]:
payload = ','.join(map(str, body))
payload

'9.0,5.0,3.8,1.5,5.9,3.0,1.5,1,0,0'

__THE WAY IT SHOULD BE__

In [80]:
# first line from the "raw" data as a request from the web form
# NOTE: This is manually constructed from the raw data
request = {"body": "{\"length\":\"0.455\",\"diameter\":\"0.365\",\"height\":\"0.095\",\"wholeweight\":\"0.514\",\"shuckedweight\":\"0.2245\",\"visceraweight\":\"0.101\",\"shellweight\":\"0.15\",\"sex\":\"M\"}"}
# request = {"body": "{\"length\":\"9.0\",\"diameter\":\"5.0\",\"height\":\"3.8\",\"wholeweight\":\"1.5\",\"shuckedweight\":\"5.9\",\"visceraweight\":\"3.0\",\"shellweight\":\"1.5\",\"female\":\"1\",\"infant\":\"0\",\"male\":\"0\"}"}

In [81]:
import json
import pandas as pd

body = json.loads(request['body'])
body

{'length': '0.455',
 'diameter': '0.365',
 'height': '0.095',
 'wholeweight': '0.514',
 'shuckedweight': '0.2245',
 'visceraweight': '0.101',
 'shellweight': '0.15',
 'sex': 'M'}

In [82]:
df = pd.json_normalize(json.loads(request['body']))
df

,length,diameter,height,wholeweight,shuckedweight,visceraweight,shellweight,sex
0,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,M


In [83]:
if df['sex'][0] == 'M':
    df['sex_f'] = 0
    df['sef_i'] = 0
    df['sex_m'] = 1
elif df['sex'][0] == 'F':
    df['sex_f'] = 1
    df['sex_i'] = 0
    df['sex_m'] = 0
elif df['sex'][0] == 'I':
    df['sex_f'] = 0
    df['sex_i'] = 1
    df['sex_m'] = 0

In [84]:
df

,length,diameter,height,wholeweight,shuckedweight,visceraweight,shellweight,sex,sex_f,sef_i,sex_m
0,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,M,0,0,1


In [85]:
df = df.drop('sex', 1)

In [86]:
df

,length,diameter,height,wholeweight,shuckedweight,visceraweight,shellweight,sex_f,sef_i,sex_m
0,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,0,0,1


In [87]:
payload = df.to_csv(header=None, index=False).strip('\n').split('\n')[0]

In [101]:
payload

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [102]:
df

,length,diameter,height,wholeweight,shuckedweight,visceraweight,shellweight,sex_f,sef_i,sex_m
0,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,0,0,1


In [103]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]
)

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)

In [104]:
x_pre = df.drop(columns=['sex_f', 'sef_i', 'sex_m'], axis=1)

In [105]:
x_pre

,length,diameter,height,wholeweight,shuckedweight,visceraweight,shellweight
0,0.455,0.365,0.095,0.514,0.2245,0.101,0.15


In [106]:
# numeric_features = [col for col in x.columns]
# x_post = preprocess.fit_transform(x_pre)

In [113]:
x_post

array([[0., 0., 0., 0., 0., 0., 0.]])

In [114]:
import numpy as np
payload = np.concatenate((x_post, [[0,1,0]]), axis=1)

In [115]:
payload

array([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [116]:
from sklearn import preprocessing
x_post = preprocessing.normalize(x_pre)

In [167]:
payload

'[0.54516543 0.43733051 0.11382575 0.61585722 0.26898822 0.12101474\n 0.17972487 0.         1.         0.        ]'

In [24]:
from sklearn import preprocessing
import numpy as np
import pandas as pd
import json


def handle_encoding(sex):
    if sex == 'M' or sex == 'm':
        return [0., 0., 1.0]
    elif sex == 'F' or sex == 'f':
        return [1.0, 0., 0.]
    elif sex == 'I' or sex == 'i':
        return [0., 1.0, 0.]



request = {"body": "{\"length\":\"0.455\",\"diameter\":\"0.365\",\"height\":\"0.095\",\"whole_weight\":\"0.514\",\"shucked_weight\":\"0.2245\",\"viscera_weight\":\"0.101\",\"shell_weight\":\"0.15\",\"sex\":\"M\"}"}

In [25]:
df = pd.json_normalize(json.loads(request['body']))
df

,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,sex
0,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,M


In [26]:
s_pre = df['sex'][0]
s_pre

'M'

In [27]:
s_post = handle_encoding(s_pre)
s_post

[0.0, 0.0, 1.0]

In [28]:
x_pre = df.drop(columns=['sex'], axis=1)
x_pre

,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight
0,0.455,0.365,0.095,0.514,0.2245,0.101,0.15


In [29]:
# x_post = preprocessing.normalize(x_pre).tolist()[0]
# x_post

In [30]:
x_post = x_pre.values.tolist()[0]
x_post

['0.455', '0.365', '0.095', '0.514', '0.2245', '0.101', '0.15']

In [31]:
# payload = np.concatenate((x_post, s_post), axis=1)
#payload = np.concatenate((x_post, [[0,1,0]]), axis=1)
payload = ','.join(map(str, x_post+s_post))

In [32]:
payload

'0.455,0.365,0.095,0.514,0.2245,0.101,0.15,0.0,0.0,1.0'